# CC3001 Otoño 2021 Tarea 3

### Profesores
Sección 1 Iván Sipirán •
Sección 2 Patricio Poblete •
Sección 3 Nelson Baloian

Pablo González Aguilera - 20973277-7
Sección 2

# El Problema de la Mochila (*Knapsack*)

El problema consiste en lo siguiente. Se tiene una mochila con capacidad para cargar un peso $T$ y un conjunto de $n$ objetos, tal que cada objeto tiene un peso y un valor: $M = \{(p_1,v_1),(p_2,v_2),\ldots, (p_n,v_n)\}$. El problema es encontrar el máximo valor que se puede cargar en la mochila usando objetos de $M$, sin sobrepasar el peso permitido $T$.

Este problema se resuelve con programación dinámica teniendo en cuenta lo siguiente:

*   Representamos con $C(i,j)$ el valor óptimo que se puede obtener con $i$ objetos y una mochila de capacidad $j$.
*   El análisis para encontrar el valor óptimo $C(i,j)$ incluye dos posibles opciones:
  *   Que el $i-$ésimo objeto sea parte del óptimo. En ese caso se esperaría que $C(i,j)$ incluya el valor $v_i$ y que la mochila sin el $i-$ésimo objeto también tenga un valor óptimo. O sea  $C(i,j) = v_i + C(i-1, j - p_i)$. Tener en cuenta que este razonamiento sólo funciona si la capacidad $j$ sobrepasa el peso $p_i$.
  *   Que el $i$-ésimo objeto no sea parte del óptimo. En este caso el óptimo $C(i,j)$ es simplemente lo mismo que el óptimo de no considerar al objeto $i$. O sea $C(i,j) = C(i-1,j)$
  
  El máximo de estas dos opciones nos dará el óptimo para $C(i,j)$.

Las condiciones básicas para resolver este problema son:


*   $C(i,0) = 0, \forall i$ (el óptimo con una mochila de capacidad 0, siempre es 0)
*   $C(0,j) = 0, \forall j$ (el óptimo de no tener ningún objeto es 0)

Como puede observarse, este problema puede resolverse con el método de tabulación. Los pasos serían los siguientes:

1.   Crear una tabla de tamaño $(n+1) \times (T+1)$.
2.   Llenar toda la primera fila y columna con ceros, para cumplir con las condiciones iniciales del problema.
3.   Recorrer la matriz llenando sus valores con la regla:

$$
C(i,j) =
\begin{cases}
max\{C(i-1,j), v_i + C(i-1,j-p_i)\} & \text{ si } j - p_i \geq 0 \\
C(i-1,j) & \text{ si } j - p_i < 0 
\end{cases}
$$

4.   Retornar la solución del problema $C(n,T)$: el elemento almacenado en la posición de más a la derecha y abajo de la tabla.

# Enunciado de la Tarea
A John le gusta bucear y cazar tesoros. Para su buena suerte, él acaba de encontrar la ubicación de
un barco pirata lleno de tesoros. El sofisticado sistema sonar a bordo de su barco le permite identificar
la ubicación, profundidad y cantidad de oro en cada tesoro hundido. Desafortunadamente, John olvidó
traer un dispositivo GPS y las posibilidades de encontrar esta ubicación de nuevo son muy escasas, por
lo que tiene que apropiarse del oro ahora. Para empeorar la situación, John solo tiene un tanque de aire
comprimido.

John quiere bucear con el tanque de aire comprimido para recuperar tanto oro como sea posible del
naufragio. Escriba un programa que John pueda usar para seleccionar cuáles tesoros debe recoger para
maximizar la cantidad de oro recuperado. El problema tiene las siguientes restricciones:

*   Existen $n$ tesoros $\{(d_1, v_1), (d_2,v_2), \ldots, (d_n, v_n)\}$ representados por el par (profundidad, cantidad de oro). Existen como máximo 30 tesoros.
*   El tanque de aire solo permite estar $T$ segundos bajo el agua. $T$ es como máximo 1000 segundos.
*   En cada inmersión, John puede traer como máximo un tesoro a la vez.
*   El tiempo para descender es $t_{di} = w*d_i$, donde $w$ es un número entero constante.
*   El tiempo para ascender es $t_{ai} = 2w*d_i$, donde $w$ es el mismo número entero constante.
*   Debido a limitaciones de los instrumentos, todos los parámetros son números enteros.

### Ejemplos. 
1. Si $T=210$, $w=4$ y los tesoros tienen profundidades y valores $\{(10,5),(10,1),(7,2)\}$, la cantidad óptima de oro es 7 escogiendo los tesoros $\{(10,5),(7,2)\}$

2. Si T=210, $w=4$ y los tesoros tienen profundidades y valores $\{(4,2),(1, 2),(1,3),(5,7),(4,4),(12,100)\}$, la cantidad óptima de oro es 107 escogiendo los tesoros $\{(5,7),(12,100)\}$

---
# Lo que usted tiene que hacer:

Implementar la función *tesoro_maximo* que recibe la lista de tesoros, la constante $w$ y la capacidad del tanque $T$, y devuelve el máximo valor del tesoro recuperado.

La función *tesoro_maximo*, que se define a continuación puede ser usada como base para desarrollar el problema. Esta función hace algo no óptimo: simplemente escoge el primer objeto que puede ser rescatado (aquel cuyo tiempo de rescate es menor a $T$), y devuelve su valor.

Modifique la función *tesoro_maximo* para solucionar el problema de John con programación dinámica. Use el problema de la mochila como referencia.

In [ ]:
import numpy as np
from numpy.lib.index_tricks import diag_indices

#tesoro_maximo: list[tupla] int int -> int
# # Retorna el valor de tesoro máximo a obtener, dados 

# L = listaTesoro
# t = tiempoOxigeno;
# W = factorTiempoSubida/Bajada

# tSubida = w*d
# tBajada = 2*w*d 

def tesoro_maximo(L, w, T):

    n = len(L)

    #Matriz de tabulación (inicializada con ceros)
    C = np.zeros(shape=(n+1,T+1),dtype=int)

    assert n <= 30 # Como máximo hay 30 tesoros
    assert T <= 1000 # Como máximo el tanque dura 1000 segundos 

    # Iteramos sobre
    for objeto in L:

        # Las tuplas en L indican la distancia absoluta (no necesariamente la recorrida), 
        # junto con el valor de cada tesoro
        (dAbs_i, v_i) = objeto[0], objeto[1]

        # t_i es el tiempo efectivo a recorrer
        t_i = 3*dAbs_i*w # Se demora 1 en bajar y 2 en subir (3 distancuas en total)

        #itero sobre los C[i,j]
        for i in range(n+1):
            for j in range(T+1):

                # Caso 1
                if  j - t_i < 0:
                    C[i,j] = C[i-1,j]

                #Caso 2 
                else:

                    #### Para debugging, cuando este listo cambiar por lo de abajo!
                    # K = C[i-1,j]
                    # N = C[i-1, j-t_i]
                    # C[i,j] = max(K, v_i + N)

                    C[i,j] = max(C[i-1,j], v_i + C[i-1, j-t_i])
                    

                # # Solo para debugging
                # M = C[i,j]
                # if M == 106:
                #     pass
                # ##########

            j = j + 1
        i = i + 1

    # Por tabulación, el optimo se encuentra en el valor en la posición C[n,T]
    return C[n,T]


# Tests
tesoros = [(10,5), (10,1),(7,2)]
tesoros1 = [(4,2),(1, 2),(1,3),(5,7),(4,4),(12,100)]

#Este funciona
#assert tesoro_maximo(tesoros, 4, 210) == 7

#Este no :(
  #da 115 en vez de 107 (se pasa)
#assert tesoro_maximo(tesoros1, 4, 210) == 107



print(tesoro_maximo(tesoros, 4, 210))
print(tesoro_maximo(tesoros1, 4, 210))


Si ejecutamos la función con los datos del primer caso de ejemplo, el resultado es:

In [ ]:
tesoros = [(10,5),(10,1),(7,2)]
maximo = tesoro_maximo(tesoros, 4, 210)

print('El máximo valor es:', maximo)


El máximo valor es: 5


## ¿Qué hay que entregar?

Usted debe entregar este mismo archivo, modificado de acuerdo a lo que se pide. Documentar adecuadamente su código. Cite todas las fuentes de información utilizadas. **No olvide poner su nombre en el encabezamiento**.